In [2]:
import ciftify
from ciftify.utilities import run
import os
import datetime
import logging

In [71]:
working_dir = '/scratch/edickie/ciftify_intergration_tests/'
src_data_dir= os.path.join(working_dir,'src_data')
work_from = None

if work_from:
    new_outputs = work_from
else:
    new_outputs= os.path.join(working_dir,'run_{}'.format(datetime.date.today()))
DEBUG = True
DRYRUN = True

## getting the data

freesurfer_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_freesurfer_sub50004-50008.zip'
func_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_sub50004-50008.zip'

#subids = ['sub-50005','sub-50007']
subids = ['sub-50004','sub-50005','sub-50006','sub-50007', 'sub-50007']

logger = logging.getLogger('ciftify')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.WARNING)
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)    

In [55]:
for subid in subids:
    sub_fs_path = os.path.join('ds000030_R1.0.4','derivatives','freesurfer', subid)
if not os.path.exists(os.path.join(src_data_dir, sub_fs_path)):
    if not os.path.exists(os.path.join(src_data_dir, 
                                       os.path.basename(freesurfer_webtgz))):
        run(['mkdir','-p',src_data_dir])
        run(['wget', '-P', src_data_dir, freesurfer_webtgz])

    run(['unzip', 
         os.path.join(src_data_dir, 
                      os.path.basename(freesurfer_webtgz)), 
         os.path.join(sub_fs_path,'*'),
         '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

In [5]:
fs_subjects_dir = os.path.join(src_data_dir, 'ds000030_R1.0.4',
                               'derivatives','freesurfer')
hcp_data_dir = os.path.join(new_outputs, 'hcp')

if not os.path.exists(hcp_data_dir):
    run(['mkdir','-p',hcp_data_dir])

In [6]:
run(['ciftify_recon_all', '--resample-LowRestoNative', '--hcp-data-dir', hcp_data_dir,
    '--fs-subjects-dir', fs_subjects_dir, 
    subids[0]])

0

In [80]:
for subid in subids:
    run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

['sub-50004', 'sub-50005', 'sub-50006', 'sub-50007', 'sub-50007']

In [56]:
for subid in ['sub-50004','sub-50006', 'sub-50008']:
    run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
        '--fs-subjects-dir', fs_subjects_dir, 
        subid])

cmd: ciftify_recon_all --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-08-25/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50004 
 Failed with returncode 1
cmd: ciftify_recon_all --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-08-25/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50004 
 Failed with returncode 1
Traceback (most recent call last):
  File "/projects/edickie/code/ciftify/ciftify/bin/ciftify_recon_all", line 1523, in <module>
    ret = main(tmpdir, settings)
  File "/projects/edickie/code/ciftify/ciftify/bin/ciftify_recon_all", line 1436, in main
    settings.grayord_res, settings.ciftify_data_dir, temp_dir)
  File "/projects/edickie/code/ciftify/ciftify/bin/ciftify_recon_all", line 1062, in create_cifti_subcortical_ROIs
    'Atlas_ROIs.{}.nii.gz'.format(grayord_res)))
  File "/projects/edickie

In [81]:
for subid in ['sub-50004','sub-50006', 'sub-50008']:
    run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

0

# Getting ABIDE

In [85]:
abide_amazon_addy = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1'

subid = 'NYU_0050954'
abide_freesurfer = os.path.join(src_data_dir, 'abide', 'freesurfer')

fs_subdir = os.path.join(abide_freesurfer, subid)
if not os.path.exists(fs_subdir):    
    run(['mkdir', '-p', fs_subdir])

for subdir in ['mri', 'surf','label','scripts']:
    run(['mkdir', '-p', os.path.join(fs_subdir, subdir)])
    

import subprocess
for filename in ['T1.mgz', 'aparc\+aseg.mgz', 'aparc.a2009s\+aseg.mgz', 'wmparc.mgz', 'brain.finalsurfs.mgz']:
    subprocess.call(['wget', os.path.join(abide_amazon_addy, subid, 'mri', filename),'-O', os.path.join(fs_subdir, 'mri', filename)],
                   shell = True)

for surface in ['pial', 'white', 'sphere.reg', 'sphere', 'curv', 'sulc', 'thickness']:
    for hemi in ['lh', 'rh']:
        run(['wget', os.path.join(abide_amazon_addy, subid, 'surf', '{}.{}'.format(hemi,surface)),
         '-O', os.path.join(fs_subdir, 'surf', '{}.{}'.format(hemi,surface))]) 

for labelname in ['aparc', 'aparc.a2009s', 'BA']:
    for hemi in ['lh', 'rh']:
        run(['wget', os.path.join(abide_amazon_addy, subid, 'label', '{}.{}.annot'.format(hemi,labelname)),
         '-O', os.path.join(fs_subdir, 'label', '{}.{}.annot'.format(hemi,labelname))])  

for script in ['recon-all.done', 'build-stamp.txt']:
    run(['wget', os.path.join(abide_amazon_addy, subid, 'scripts', script),
         '-O', os.path.join(fs_subdir, 'scripts', script)])

--2017-08-25 16:43:41--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/surf/lh.pial
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.230.173
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.230.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3727201 (3.6M) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/surf/lh.pial’

     0K .......... .......... .......... .......... ..........  1%  372K 10s
    50K .......... .......... .......... .......... ..........  2%  854K 7s
   100K .......... .......... .......... .......... ..........  4% 1.37M 5s
   150K .......... .......... .......... .......... ..........  5% 1.28M 5s
   200K .......... .......... .......... .......... ..........  6% 1.24M 4s
   250K .......... .......... .......... .......... ..........  8% 1.16M 4s
   300K .......... .......... .......... ..........

--2017-08-25 16:43:41--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/surf/lh.pial
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.230.173
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.230.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3727201 (3.6M) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/surf/lh.pial’

     0K .......... .......... .......... .......... ..........  1%  372K 10s
    50K .......... .......... .......... .......... ..........  2%  854K 7s
   100K .......... .......... .......... .......... ..........  4% 1.37M 5s
   150K .......... .......... .......... .......... ..........  5% 1.28M 5s
   200K .......... .......... .......... .......... ..........  6% 1.24M 4s
   250K .......... .......... .......... .......... ..........  8% 1.16M 4s
   300K .......... .......... .......... ..........

--2017-08-25 16:43:48--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/surf/rh.pial
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.160.189
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.160.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3694981 (3.5M) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/surf/rh.pial’

     0K .......... .......... .......... .......... ..........  1%  190K 19s
    50K .......... .......... .......... .......... ..........  2%  303K 15s
   100K .......... .......... .......... .......... ..........  4%  242K 15s
   150K .......... .......... .......... .......... ..........  5%  265K 14s
   200K .......... .......... .......... .......... ..........  6%  369K 13s
   250K .......... .......... .......... .......... ..........  8%  459K 12s
   300K .......... .......... .......... .....

--2017-08-25 16:43:56--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/surf/lh.white
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.64.19
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.64.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3727206 (3.6M) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/surf/lh.white’

     0K .......... .......... .......... .......... ..........  1%  447K 8s
    50K .......... .......... .......... .......... ..........  2% 1.43M 5s
   100K .......... .......... .......... .......... ..........  4%  917K 5s
   150K .......... .......... .......... .......... ..........  5% 1.66M 4s
   200K .......... .......... .......... .......... ..........  6% 1.66M 4s
   250K .......... .......... .......... .......... ..........  8% 6.78M 3s
   300K .......... .......... .......... .......... ..

--2017-08-25 16:43:56--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/surf/lh.white
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.64.19
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.64.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3727206 (3.6M) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/surf/lh.white’

     0K .......... .......... .......... .......... ..........  1%  447K 8s
    50K .......... .......... .......... .......... ..........  2% 1.43M 5s
   100K .......... .......... .......... .......... ..........  4%  917K 5s
   150K .......... .......... .......... .......... ..........  5% 1.66M 4s
   200K .......... .......... .......... .......... ..........  6% 1.66M 4s
   250K .......... .......... .......... .......... ..........  8% 6.78M 3s
   300K .......... .......... .......... .......... ..

...

--2017-08-25 16:44:58--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/surf/lh.thickness
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.32.171
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.32.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 414003 (404K) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/surf/lh.thickness’

     0K .......... .......... .......... .......... .......... 12%  435K 1s
    50K .......... .......... .......... .......... .......... 24%  559K 1s
   100K .......... .......... .......... .......... .......... 37% 1.58M 0s
   150K .......... .......... .......... .......... .......... 49% 1.10M 0s
   200K .......... .......... .......... .......... .......... 61% 1.16M 0s
   250K .......... .......... .......... .......... .......... 74% 1.05M 0s
   300K .......... .......... .......... ....

--2017-08-25 16:45:11--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/label/lh.aparc.annot
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.114.0
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.114.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 829490 (810K) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/label/lh.aparc.annot’

     0K .......... .......... .......... .......... ..........  6%  503K 2s
    50K .......... .......... .......... .......... .......... 12% 1.65M 1s
   100K .......... .......... .......... .......... .......... 18%  865K 1s
   150K .......... .......... .......... .......... .......... 24% 1.69M 1s
   200K .......... .......... .......... .......... .......... 30%  120M 0s
   250K .......... .......... .......... .......... .......... 37% 1.69M 0s
   300K .......... .......... .......... 

--2017-08-25 16:45:22--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/label/lh.aparc.a2009s.annot
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.121.42
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.121.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 831300 (812K) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/label/lh.aparc.a2009s.annot’

     0K .......... .......... .......... .......... ..........  6%  394K 2s
    50K .......... .......... .......... .......... .......... 12%  662K 1s
   100K .......... .......... .......... .......... .......... 18% 1.21M 1s
   150K .......... .......... .......... .......... .......... 24% 1.27M 1s
   200K .......... .......... .......... .......... .......... 30% 96.6M 1s
   250K .......... .......... .......... .......... .......... 36% 1.68M 1s
   300K .......... ......

--2017-08-25 16:45:34--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/label/lh.BA.annot
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.230.205
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.230.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 828428 (809K) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/label/lh.BA.annot’

     0K .......... .......... .......... .......... ..........  6%  557K 1s
    50K .......... .......... .......... .......... .......... 12%  836K 1s
   100K .......... .......... .......... .......... .......... 18% 1.62M 1s
   150K .......... .......... .......... .......... .......... 24%  838K 1s
   200K .......... .......... .......... .......... .......... 30%  107M 1s
   250K .......... .......... .......... .......... .......... 37% 1.63M 0s
   300K .......... .......... .......... ..

--2017-08-25 16:45:39--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/label/rh.BA.annot
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.66.27
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.66.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 821268 (802K) [application/octet-stream]
Saving to: ‘/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/label/rh.BA.annot’

     0K .......... .......... .......... .......... ..........  6%  563K 1s
    50K .......... .......... .......... .......... .......... 12%  854K 1s
   100K .......... .......... .......... .......... .......... 18% 1.66M 1s
   150K .......... .......... .......... .......... .......... 24% 1.65M 1s
   200K .......... .......... .......... .......... .......... 31% 1.66M 1s
   250K .......... .......... .......... .......... .......... 37% 1.67M 0s
   300K .......... .......... .......... ......

In [84]:
run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
        '--fs-subjects-dir', abide_freesurfer, 
        'NYU_0050954'])

cmd: ciftify_recon_all --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-08-25/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer NYU_0050954 
 Failed with returncode 1
cmd: ciftify_recon_all --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-08-25/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer NYU_0050954 
 Failed with returncode 1
cmd: ciftify_recon_all --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-08-25/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer NYU_0050954 
 Failed with returncode 1
/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc.a2009s+aseg.mgz not found
/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz not found
cmd: mris_convert -c /scratch/edickie/ciftify_intergration_tests/src_data/abide/freesur

1

In [ ]:
pipeline="cpac"
strategy="filt_global"
site="Pitt"
src_vmhc = os.path.join(src_data_dir, 'abide', 'vmhc')

subjects=['Pitt_0050002','Pitt_OO50005']

sub_vmhc = os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subid))
mkdir -p ${sourcedir}
cd ${sourcedir}
for subject in ${subjects}; do
wget https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/${pipeline}/${strategy}/vmhc/${site}_00${subject}_vmhc.nii.gz
done

In [8]:
subid=subids[0]
native_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_preproc.nii.gz'.format(subid))
mat_file = os.path.join(src_data_dir, 'ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_T1_to_EPI.mat'.format(subid))

with open(mat_file, "w") as text_file:
    text_file.write('''1.03  -0.015  0.0025  -15.0
0.014  1.01  -0.005  -11.9
-0.007 0.01  0.99  2
0  0  0  1
''')

t1_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
if not os.path.exists(native_func):
    run(['flirt', '-in', t1_func, '-ref', t1_func, 
         '-out', native_func, '-init', mat_file, '-applyxfm'])

run(['ciftify_subject_fmri', '--SmoothingFWHM', '12', 
    '--hcp-data-dir', hcp_data_dir, 
    native_func, subid, 'rest_test1'])

0

In [27]:
subid=subids[0]
run(['cifti_vis_fmri', 'snaps', '--SmoothingFWHM', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test1', subid])

0

In [9]:
native_func_mean = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_mean.nii.gz'.format(subid))

run(['fslmaths', native_func, '-Tmean', native_func_mean])

run(['ciftify_subject_fmri', '--hcp-data-dir', hcp_data_dir, 
    '--FLIRT-template', native_func_mean, native_func, subid, 'rest_test2'])


0

In [28]:
run(['cifti_vis_fmri', 'snaps', '--smooth-conn', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test2', subid])

0

In [10]:
subid = subids[1]
run(['ciftify_subject_fmri', '--SmoothingFWHM', '8', 
    '--hcp-data-dir', hcp_data_dir, 
     '--OutputSurfDiagnostics',
    os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid)),
    subid, 'rest_test1'])

0

In [29]:
subid = subids[1]
run(['cifti_vis_fmri', 'snaps', '--hcp-data-dir',hcp_data_dir, 'rest_test1', subid])

0

In [30]:
run(['cifti_vis_fmri', 'index', '--hcp-data-dir', hcp_data_dir])

0

In [32]:
from glob import glob
dtseries_files = glob(os.path.join(hcp_data_dir,'*',
                                       'MNINonLinear', 'Results',
                                       'rest_test1', '*Atlas_s*.dtseries.nii'))

for dtseries in dtseries_files:
    run(['cifti_vis_RSN', 'cifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, dtseries, 
         os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(dtseries)))))])
run(['cifti_vis_RSN', 'index', 
     '--hcp-data-dir', hcp_data_dir])

0

In [33]:
subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--pcorr', 
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])


0

In [34]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

0

In [35]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])

0

In [36]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

0

In [37]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--outputall',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, '{}_all'.format(subid))])

0

In [38]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_all_summary.csv'.format(subid))])

0

In [39]:
run(['cifti_vis_PINT', 'index', '--hcp-data-dir',  hcp_data_dir])

0

In [40]:
summary_files = glob(os.path.join(new_outputs, 'PINT', '*','*_summary.csv'))
concat_cmd = ['ciftify_postPINT1_concat.py',  
           os.path.join(new_outputs, 'PINT', 'concatenated.csv')]
concat_cmd.extend(summary_files)
run(concat_cmd)

0

In [41]:
run(['ciftify_postPINT2_sub2sub.py', os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances.csv')])

0

In [42]:
run(['ciftify_postPINT2_sub2sub.py', '--roiidx', '14',
     os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances_roiidx14.csv')])

0

In [43]:
vertices1_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices1_csv, "w") as text_file:
    text_file.write('''hemi,vertex
L,11801
L,26245
L,26235
L,26257
L,13356
L,289
L,13336
L,13337
L,26269
L,13323
L,26204
L,26214
L,13326
L,13085
L,13310
L,13281
L,13394
L,13395
L,26263
L,26265
L,13343
L,77
L,13273
L,13342
L,26271
L,11804
L,13322
L,13369
L,13353
L,26268
L,26201
L,26269
L,68
L,13391
''')

subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '10', '--gaussian',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

0

In [44]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     subid, 
     'gaussian_roi'])

0

In [45]:
vertices2_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices2_csv, "w") as text_file:
    text_file.write('''hemi,vertex
R,2379
R,2423
R,2423
R,2629
R,29290
R,29290
R,1794
R,29199
R,1788
R,2380
R,2288
R,29320
R,29274
R,29272
R,29331
R,29356
R,2510
R,29345
R,2506
R,2506
R,1790
R,29305
R,2630
R,2551
R,2334
R,2334
R,29306
R,2551
R,2422
R,29256
R,2468
R,29332
R,2425
R,29356
''')
    
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '6', '--probmap',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

0

In [46]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     subid, 
     'probmap_roi'])

0

In [47]:
run(['ciftify_surface_rois', '--debug',
      os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
      '6', '--vertex-col', 'tvertex', '--labels-col', 'NETWORK', '--overlap-logic', 'EXCLUDE',
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

0

In [48]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     subid, 
     'tvertex'])

0

In [49]:
subid = subids[0]
smoothing = 12
func_vol = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1.nii.gz')
func_cifti_sm0 = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s0.dtseries.nii')
func_cifti_smoothed = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s{}.dtseries.nii'.format(smoothing))
atlas_vol =  os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'wmparc.nii.gz')

seed_corr_dir = os.path.join(new_outputs, 'ciftify_seed_corr')
if not os.path.exists(seed_corr_dir):
    run(['mkdir', '-p', seed_corr_dir])
struct = 'RIGHT-PUTAMEN'
putamen_vol_seed_mask = os.path.join(seed_corr_dir, '{}_{}_vol.nii.gz'.format(subid, struct))
run(['wb_command', '-volume-label-to-roi', atlas_vol, putamen_vol_seed_mask, '-name', struct])

0

In [50]:
run(['ciftify_seed_corr', func_vol, putamen_vol_seed_mask])

0

In [11]:
def seed_corr_default_out(func, seed):
    functype, funcbase = ciftify.utilities.determine_filetype(func)
    _, seedbase = ciftify.utilities.determine_filetype(seed)
    outputdir = os.path.dirname(func)
    outbase = '{}_{}'.format(funcbase, seedbase)
    outputname = os.path.join(outputdir, outbase)
    if functype == "nifti": outputname = '{}.nii.gz'.format(outputname)
    if functype == "cifti": outputname = '{}.dscalar.nii'.format(outputname)
    return(outputname)

In [ ]:
run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_vol, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitonifti_unmasked'.format(subid, struct)])

In [ ]:
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, putamen_vol_seed_mask])

In [ ]:
run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     subid, 
     '{}_{}_niftitonifti_masked'.format(subid, struct)])

In [ ]:
run(['ciftify_seed_corr', func_cifti_smoothed, putamen_vol_seed_mask])

In [ ]:
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_cifti_smoothed, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitocifti_unmasked'.format(subid, struct)])

In [ ]:
cifti_mask = os.path.join(seed_corr_dir, '{}_func_mask.dscalar.nii'.format(subid))
run(['wb_command', '-cifti-math', "'(x > 0)'", cifti_mask,
    '-var', 'x', func_cifti_sm0, '-select', '1', '1'])
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     '--mask', cifti_mask,
     func_cifti_smoothed, putamen_vol_seed_mask])

In [ ]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     subid, 
     '{}_{}_niftitocifti_masked'.format(subid, struct)])

In [ ]:
result_map = seed_corr_default_out(func_cifti_smoothed, 
                                   os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct)))
result_prefix = '{}_{}_ciftitocifti_unmasked'.format(subid, struct)
result_type = 'cifti'

In [ ]:
run(['wb_command', '-cifti-create-dense-from-template',
     func_cifti_sm0,
     os.path.join(seed_corr_dir, 
                                 '{}_{}_cifti.dscalar.nii'.format(subid, struct)),
     '-volume-all', putamen_vol_seed_mask])

run(['ciftify_seed_corr', func_cifti_smoothed, 
    os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))])

In [31]:
def run_vis_map(result_map, result_prefix, result_type):
    run(['cifti_vis_map', '{}-snaps'.format(result_type), '--hcp-data-dir', hcp_data_dir, 
     result_map, subid, result_prefix])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_{}_ciftitocifti_masked.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_masked'.format(subid, struct)

In [ ]:
run(['ciftify_seed_corr',     
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
    os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_gaussian_ciftitocifti_unmasked'.format(subid)


In [ ]:
run(['ciftify_seed_corr', '--weighted', '--surfaceonly', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_gaussian_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = os.path.join(seed_corr_dir, '{}_gaussian_ciftitocifti_masked'.format(subid))

In [ ]:
run(['ciftify_seed_corr', '--weighted', '--surfaceonly', 
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
L_gaussian_roi = os.path.join(new_outputs, 'rois', 'gaussian_L_roi.shape.gii')

result_map = seed_corr_default_out(func_cifti_smoothed, L_gaussian_roi)
result_type = 'cifti'
result_prefix = '{}_gaussian_giftitocifti_unmasked'.format(subid)


run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_LEFT', L_gaussian_roi])
run(['ciftify_seed_corr', '--weighted', '--hemi', 'L', 
     func_cifti_smoothed, 
     L_gaussian_roi])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid)

In [ ]:
run(['ciftify_seed_corr', '--weighted', '--hemi', 'L', 
     '--outputname', os.path.join(seed_corr_dir, '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     L_gaussian_roi])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_promap_ciftitocifti_unmasked.dscalar.nii'.format(subid)

In [ ]:
run(['ciftify_seed_corr', '--weighted', '--surfaceonly', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_promap_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_promap_ciftitocifti_masked.dscalar.nii'.format(subid)

In [ ]:
run(['ciftify_seed_corr', '--weighted', 
     '--surfaceonly', 
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
R_probmap_roi = os.path.join(new_outputs, 'rois', 'probmap_R_roi.shape.gii')
result_map = seed_corr_default_out(func_cifti_smoothed, R_probmap_roi) 
result_type = 'cifti'
result_prefix = '{}_promap_giftitocifti_unmasked.dscalar.nii'.format(subid)
                                   
run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_probmap_roi])
run(['ciftify_seed_corr', '--weighted', '--hemi', 'R', func_cifti_smoothed, 
     R_probmap_roi])

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid)

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
run(['ciftify_seed_corr', '--weighted', '--hemi', 'R', 
     '--outputname', os.path.join(seed_corr_dir, '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask,    
     func_cifti_smoothed, 
     R_probmap_roi])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_tvertex7_ciftitocifti_unmasked.dscalar.nii'.format(subid)

In [ ]:
run(['ciftify_seed_corr', '--roi-label', '7', '--surfaceonly', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_tvertex7_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_tvertex7_ciftitocifti_masked.dscalar.nii'.format(subid)

In [ ]:
run(['ciftify_seed_corr', '--roi-label', '7', 
     '--surfaceonly', 
     '--outputname', result_map,
     '--mask', cifti_mask, 
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
R_tvertex_roi = os.path.join(new_outputs, 'rois', 'tvertex_R_roi.shape.gii')
result_map = seed_corr_default_out(func_cifti_smoothed, R_tvertex_roi)
result_type = 'cifti'
result_prefix = '{}_tvertex7_giftitocifti_umasked.dscalar.nii'.format(subid)


run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_tvertex_roi])
run(['ciftify_seed_corr', '--roi-label', '7', '--hemi', 'R', 
     func_cifti_smoothed, R_tvertex_roi])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'meants'
result_prefix = '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid)

In [ ]:
run(['ciftify_seed_corr', '--roi-label', '7', '--hemi', 'R', 
     '--outputname', os.path.join(seed_corr_dir, '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask, 
     func_cifti_smoothed, R_tvertex_roi])

In [ ]:
run_vis_map(result_map, result_prefix, result_type)

In [ ]:
subject_aparc = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.aparc.32k_fs_LR.dlabel.nii'.format(subid))
subject_thickness = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.thickness.32k_fs_LR.dscalar.nii'.format(subid))
run(['ciftify_meants', '--surfaceonly', func_cifti_sm0, subject_aparc])


In [ ]:
run(['ciftify_meants', '--surfaceonly',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_aparc_func_ciftitocifti_masked_meants.csv'.format(subid)),
     '--mask', cifti_mask, 
     func_cifti_sm0, subject_aparc])

In [ ]:
run(['ciftify_meants', '--surfaceonly', 
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_aparc_thickness_ciftitocifti_unmasked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                    '{}_aparc_ciftitocifti_unmasked_labels.csv'.format(subid)),
         subject_thickness, subject_aparc])

In [ ]:
run(['ciftify_meants', '--surfaceonly', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_aparc_thickness_ciftitocifti_unmasked_meants.csv'.format(subid)),
         subject_thickness, subject_aparc])

In [ ]:
run(['ciftify_meants', '--surfaceonly', func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

In [ ]:
run(['ciftify_meants', '--surfaceonly',
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_tvertex_func_ciftitocifti_unmasked_meants.csv'.format(subid)),
     func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

In [ ]:
## project wmparc to subject
wmparc_dscalar = os.path.join(seed_corr_dir, '{}_wmparc_MNI.dscalar.nii'.format(subid))
run(['ciftify_vol_result', '--hcp-data-dir', hcp_data_dir,
    '--integer-labels', subid, atlas_vol, wmparc_dscalar])

In [ ]:
run(['ciftify_meants', 
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, atlas_vol])

In [ ]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dscalar])

In [ ]:
wmparc_dlabel = os.path.join(seed_corr_dir, '{}_wmparc_MNI.dlabel.nii'.format(subid))
run(['wb_command', '-cifti-label-import', '-logging', 'SEVERE',
     wmparc_dscalar, 
     os.path.join(ciftify.config.find_ciftify_global(), 'hcp_config', 'FreeSurferAllLut.txt'),
    wmparc_dlabel])
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dlabel])

In [ ]:
run(['cifti_vis_map', 'index', '--hcp-data-dir', hcp_data_dir])

In [ ]:
### All results
# ciftify_peaktable
# cifti_vis_map



In [ ]:
group_nifti = '/home/edickie/Documents/ciftify_tests/src_data/working_memory_pFgA_z_FDR_0.01.nii.gz'
group_cifti = '/home/edickie/Documents/ciftify_tests/run_2017-08-17/working_memory_pFgA_z_FDR_0.01.dscalar.nii'
run(['ciftify_vol_result', '--debug', 'HCP_S1200_GroupAvg', group_nifti, group_cifti])